<a href="https://colab.research.google.com/github/Kheri-C/ML/blob/main/Chess_and_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aprendizaje automático

Equipo 3

Kheri Saúl Chavira Leal

Luis Arturo Dan Fong

Miguel Angel Espinosa Varela

Jorge Ramírez Sánchez

Christian Rodrigo Velasco Bautista


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
path = 'https://raw.githubusercontent.com/Kheri-C/ML/main/chess_games.csv'
df = pd.read_csv(path)
df = df.drop(["game_id","white_id","moves","black_id","opening_code","opening_fullname","opening_shortname","opening_response","opening_variation","time_increment"],axis="columns")
df.head()

,rated,turns,victory_status,winner,white_rating,black_rating,opening_moves
0,False,13,Out of Time,White,1500,1191,5
1,True,16,Resign,Black,1322,1261,4
2,True,61,Mate,White,1496,1500,3
3,True,61,Mate,White,1439,1454,3
4,True,95,Mate,White,1523,1469,5


In [ ]:
df = df.replace(False,0) # Convertimos el flotante a número
# Conseguimos los dummies de victory_status
df = df.merge(pd.get_dummies(df["victory_status"]),left_index=True,right_index=True)
df = df.drop("victory_status",axis="columns")
df.head()
df

,rated,turns,winner,white_rating,black_rating,opening_moves,Draw,Mate,Out of Time,Resign
0,0.0,13,White,1500,1191,5,0,0,1,0
1,1.0,16,Black,1322,1261,4,0,0,0,1
2,1.0,61,White,1496,1500,3,0,1,0,0
3,1.0,61,White,1439,1454,3,0,1,0,0
4,1.0,95,White,1523,1469,5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
20053,1.0,24,White,1691,1220,2,0,0,0,1
20054,1.0,82,Black,1233,1196,2,0,1,0,0
20055,1.0,35,White,1219,1286,3,0,1,0,0
20056,1.0,109,White,1360,1227,4,0,0,0,1


In [ ]:
X = df.drop(["winner"],axis="columns")
y = df["winner"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score
algoritmo = RandomForestClassifier()
#Entreno el modelo
algoritmo.fit(X_train, y_train)
#Realizo una predicción
y_pred = algoritmo.predict(X_test)
#Calculo la precisión del modelo
precision = precision_score(y_test, y_pred, average='micro')
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6969092721834497


**KFold**

In [ ]:
from sklearn.model_selection import KFold
dfs = []
kf = KFold(n_splits=3, shuffle=True, random_state=123)
i = 1

for train_index, test_index in kf.split(df):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.loc[test_index]
    
    #Train the model
    algoritmo.fit(X_train, y_train) #Training the model
    y_pred = algoritmo.predict(X_test)
    print(f"Accuracy for the fold no. {i} on the test set: {accuracy_score(y_test, y_pred)}")

Accuracy for the fold no. 1 on the test set: 0.6889021836673647
Accuracy for the fold no. 1 on the test set: 0.6881543523781035
Accuracy for the fold no. 1 on the test set: 0.6921926413401137


In [ ]:
from pprint import pprint
pprint(algoritmo.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


**GRID SEARCH**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [80],#[80, 90, 100, 110],
    'max_features': [3],#[2, 3],
    'min_samples_leaf': [3],#[3,4,5],
    'min_samples_split': [10],#[8, 10, 12],
    'n_estimators': [1000]#[100, 200, 300, 1000]
}

# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, scoring='accuracy', verbose = 1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  4.6min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [ ]:
print('Cross Validation Score:', grid_search.best_score_)
# Best parameters which resulted in the best score
print('Best Parameters:', grid_search.best_params_)

Cross Validation Score: 0.6882296954403428
Best Parameters: {'bootstrap': True, 'max_depth': 110, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 1000}


In [ ]:
# Predict with X_test features
y_pred = grid_search.predict(X_test)

# Compare predictions to y_test labels
test_score = accuracy_score(y_test, y_pred)
print('Accuracy Score on test data set:', test_score)

Accuracy Score on test data set: 0.6948848339814537
